In [1]:
import os
import re
from pathlib import Path
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [2]:
# Parameter
input_dir = Path("data/processed")
chunk_size = 200
chunk_overlap = 0
index_path = "faiss_index"
model_name = "sentence-transformers/all-MiniLM-L6-v2"

In [3]:
# Text splitter konfigurieren
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [4]:
# Alle Dokumente sammeln
all_documents = []

for md_file in input_dir.glob("*.md"):

    # Dateiname analysieren
    filename = md_file.stem  # z. B. "bmw_2023_filtered"
    parts = filename.split("_")  # → ["bmw", "2023", "filtered"]

    # Metadaten extrahieren
    company = parts[0]
    year = parts[1]

    with md_file.open("r", encoding="utf-8") as f:
        text = f.read()

    text = text.replace('\u202f', ' ') # Filter 1: Unicode entfernen
    text = re.sub(r'(?<=\d)\.(?=\d)', '', text) # Filter 2: Entferne Punkte zwischen zwei Ziffern (z. B. 1.234 → 1234)

    chunks = text_splitter.split_text(text)
    
    for i, chunk in enumerate(chunks):
        all_documents.append(Document(
            page_content=chunk,
            metadata={"company": company, "year": year, "chunk": i}
        ))

In [5]:
bmw_chunks = [doc for doc in all_documents if doc.metadata["company"].lower() == "bmw"]
bmw_chunks[-20:]

[Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 9999}, page_content='Die Produktion von Motorrädern umfasste 221988 Einheiten.\nDer Vertragsbestand im Finanzdienstleistungsbereich lag bei 5312689 Verträgen.\nDas finanzielle Geschäftsvolumen betrug 137910 Mio. €.'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 10000}, page_content='Die Umsatzerlöse erreichten 155498 Mio. €.\nDie Bruttomarge belief sich auf 19,1 %.\nDas Ergebnis vor Finanzergebnis lag bei 18482 Mio. €.\nDas Ergebnis vor Steuern betrug 17096 Mio. €.'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 10001}, page_content='Die Umsatzrendite (Ergebnis vor Steuern/Umsatzerlöse) erreichte 11,0 %.\nDie Ertragssteuern beliefen sich auf 4931 Mio. €.\nDie Steuerquote betrug 28,8 %.'),
 Document(metadata={'company': 'BMW', 'year': '2023', 'chunk': 10002}, page_content='Die langfristigen Vermögenswerte beliefen sich auf 155918 Millionen Euro.\nDie kurzfristigen Vermögenswerte betrugen 

In [6]:
# Embeddings vorbereiten
embeddings = HuggingFaceEmbeddings(model_name=model_name)

/var/folders/9q/9wk8_fhx2zgbpf5xybsmn49w0000gn/T/ipykernel_2328/2710609826.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/Users/yanoothan/Master/Advanced NLP/BWLLM/BWLLM_venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# FAISS-Index erstellen
faiss_index = FAISS.from_documents(all_documents, embedding=embeddings)

In [9]:
# Lokal speichern
faiss_index.save_local(index_path)